In [ ]:
!unzip /content/challenges-in-representation-learning-facial-expression-recognition-challenge.zip

In [ ]:
import torch
from torch import nn, save, load
from torch.optim import Adam
import torchvision
from torchvision import transforms

In [ ]:
#this is for data preprocessing and loading with train data
def train_pl():
    #the transformation we will apply to the images from the FER2013 dataset
    transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.ToTensor(), # Convert image to tensor
        transforms.Normalize(0.485, 0.229) # Normalize image
    ])

    # loading the data from the directory I have stored the downloaded FER2013 dataset
    train_data = torchvision.datasets.FER2013(root='/content', split = 'train', transform=transform)
    print(f"Length of train data: {len(train_data)}")
    # create dataloaders so that the FER2013 data can be loaded into the model we will implement
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=19, shuffle=True, num_workers=2)

    return train_loader

#this is for data preprocessing and loading with test data
def test_pl():
    #the transformation we will apply to the images from the FER2013 dataset
    transform = transforms.Compose([
        # transforms.Grayscale(),
        transforms.ToTensor(), # Convert image to tensor
        transforms.Normalize(0.485, 0.229) # Normalize image
    ])

    # loading the data from the directory I have stored the downloaded FER2013 dataset
    test_data = torchvision.datasets.FER2013(root='/content', split = 'test' ,  transform=transform)
    print(f"Length of test data: {len(test_data)}")
    # create dataloaders so that the FER2013 data can be loaded into the model we will implement
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False, num_workers=2)

    return test_loader

In [ ]:
class EmotionModel(nn.Module):
    def __init__(self):
        super(EmotionModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 6 * 6, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 7)  # 7 classes for different emotions
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

In [ ]:
def train():
    model = EmotionModel().to('cuda')
    optimizer = Adam(model.parameters(), lr = 1e-3)
    loss_fn = nn.CrossEntropyLoss()

    train_set = train_pl()

    for epoch in range(50): #train for 50 epochs
        for batch in train_set:
            X, y = batch
            X, y = X.to('cuda'), y.to('cuda')
            prediction = model(X)
            loss = loss_fn(prediction, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}\n-------------------------------")
        print(f"\tloss:{loss}")
        print("--------------------------------------------")

    # saving our model to our environment

    return model

In [ ]:
trained_model = train()
save(trained_model, 'model_MK1')